In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import tslearn
import os
import tslearn.utils 
import seaborn
import pylab
from tslearn.clustering import TimeSeriesKMeans, KShape
import matplotlib.ticker as ticker
import zipfile

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\tslearn\clustering\kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


In [2]:
points_path = "./iHUB - CoM - Library at the Dock/CSV_Lib@Dock/eletrical meter/"
# points_path = "./iHUB - CoM - Library at the Dock/CSV_Lib@Dock/temp/"
raw_data = []
for zip_filename in os.listdir(points_path):
    zip_path = os.path.join(points_path, zip_filename)
    zip_obj = zipfile.ZipFile(zip_path)
    csv_file = zip_obj.open('E.R. Energy Usage Hour kWh.csv')
    # remove first line
    csv_file.readline()
    
    raw_data.append(pd.read_csv(csv_file))

In [3]:
def preprocessing(meter, interpolate=True):
    t = list(meter.columns)
    t[0] = "Time stamp"
    meter.columns = t
    meter_preprocessed = meter[["Time stamp", "Value"]].dropna(axis=0, how='any')
    # meter_preprocessed["Value"] = meter_preprocessed["Value"].apply(lambda s: float(s.replace(",", "")))
    meter_preprocessed["Time stamp"] = pd.to_datetime(meter_preprocessed["Time stamp"], format="%d/%m/%Y %I:%M:%S %p %Z")
    meter_preprocessed = meter_preprocessed[meter_preprocessed["Value"] < 1000]
  
    meter_preprocessed = meter_preprocessed[["Time stamp", "Value"]].dropna(axis=0, how='any')
    if interpolate:
        beg= meter_preprocessed["Time stamp"].min().replace(hour=0, minute=59, second=57)
        end = meter_preprocessed["Time stamp"].max().replace(hour=23, minute=59, second=57)
        helper = pd.DataFrame({"Time stamp": pd.date_range(beg, end, freq='1H')})
        meter_preprocessed = pd.merge(meter_preprocessed, helper, on='Time stamp', how='outer').sort_values('Time stamp')    
        meter_preprocessed = pd.merge(meter_preprocessed, helper, on='Time stamp', how='inner').sort_values('Time stamp')
        meter_preprocessed['Value'] = meter_preprocessed['Value'].interpolate(method='linear')
        meter_preprocessed = meter_preprocessed[meter_preprocessed["Time stamp"] > beg] 
        meter_preprocessed = meter_preprocessed[(meter_preprocessed["Time stamp"] < end)] 
    meter_preprocessed = meter_preprocessed.drop_duplicates(subset=["Time stamp"])
    return meter_preprocessed

In [4]:
preprocessed_data = [preprocessing(x, False) for x in raw_data]

for x in preprocessed_data: 
    print(len(x))

7235
7235
7235
7239
7239
7232


In [5]:
grouped_data = [x.groupby(pd.Grouper(key="Time stamp", freq="D")) for x in preprocessed_data]
for x in grouped_data:
    print(len(x))
    break

366


In [6]:
grouped_data_filtered = [x.filter(lambda x: len(x) > 23) for x in grouped_data]

for temp in range(len(grouped_data_filtered)):
    if temp == 0:
        continue
    out = pd.merge(grouped_data_filtered[0], grouped_data_filtered[temp], how="right", on="Time stamp")
    out = out.dropna(axis=0, how='any')[["Time stamp", "Value_y"]]
    out.columns = ["Time stamp", "Value"]
    grouped_data_filtered[temp] = out

In [7]:
for temp in range(len(grouped_data_filtered)):
    grouped_data_filtered[temp] = grouped_data_filtered[temp].set_index("Time stamp")

In [8]:
val = sum([x["Value"] for x in grouped_data_filtered])
print(len(val))

6912


In [9]:
meter_sum = pd.DataFrame({"Value": sum([x["Value"] for x in grouped_data_filtered])})
meter_sum.to_csv("./library.csv")